## Adidas - Dataset

#### Inicializacion
Primero, inicializamos todas nuestras librerías por usar. En este caso, utilizaremos Selenium y BeautifulSoup para poder manejar todo el HTML que vamos a descargar de la fuente de información. A su vez, tenemos declaraciones fundamentales como lo es el path hacia el driver del navegador y la inicialización del arreglo donde irán todos nuestros zapatos/tenis obtenidos.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time

path = "C:\\Users\\angel\\Desktop\\8avo Semestre\\Ciencia de Datos\\AdidasDataset\\chromedriver-win64\\chromedriver.exe"

# Crear una lista para almacenar solo el contenido del DIV de cada elemento
shoes_ids = []
shoes_names = []
shoes_prices = []
shoes_category = []
shoes_nColors = []
shoes_gender = []

C:\Users\angel\AppData\Local\Temp\ipykernel_8288\3479422961.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### Iniciando el driver

Ahora tenemos el siguiente código, el cual arrancará el driver para usarlo a continuación para el manejo de la ventana y la obtención de los datos con BeautifulSoup. Adicionalmente, debemos quitar una ventana emergente, por lo que usamos las funciones del driver para conseguirlo.

IMPORTANTE: Tenemos la declaración del sitio, la cual usaremos en 2 ocasiones, debido a que el apartado de todos los tenis de hombre y de mujer está en apartados diferentes.

In [7]:
# website = "https://www.adidas.mx/calzado-hombre"
website = "https://www.adidas.mx/calzado-mujer"

# gender = "Hombre"
gender = "Mujer"

In [8]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)
driver.get(website)
time.sleep(1)

# Buscar el botón "Siguiente"
next_button = driver.find_element(By.ID, 'glass-gdpr-default-consent-accept-button')
# Hacer clic en el botón "Siguiente"
next_button.click()
# Esperar un breve período de tiempo para que se cargue la siguiente página

#### Obteniendo los datos, Hombre y Mujer
El siguiente código se ejecuta tanto para el enlace al apartado de hombres como al de mujeres, extrayendo así la máxima cantidad de información posible de la página oficial de Adidas. En este, se lleva a cabo un ciclo que recorre todas las páginas disponibles del sitio indicado y extrae la información de los tenis indicando mediante Selenium de qué elementos extraerlos. Todo se va guardando en el arreglo de shoes_html para procesarlo posteriormente con BeautifulSoup.

In [9]:
# Lista con todos los tenis de nuesta fuente, primera iteracion 'Hombres', Segunda iteracion 'Mujeres'
shoes_html = []

try:    
    # Definir la cantidad de desplazamientos y el tiempo de espera entre cada uno
    scrolls = 4
    variableHeight = 2000

    # Iniciar el bucle para hacer clic en el botón "Siguiente" hasta que no esté presente
    while True:
        try:
            # Buscar el botón "Siguiente"
            adiClub_button = driver.find_element(By.ID, 'gl-modal__close-mf-account-portal')
            # Hacer clic en el botón "Siguiente"
            adiClub_button.click()
        except Exception as e:
            pass


        driver.execute_script("return document.body.scrollHeight")
        for i in range(scrolls):
            # Desplazarse hacia abajo
            driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight + {i * variableHeight});")
            # Esperar un breve período de tiempo
            time.sleep(1)
            # Obtener la nueva altura de la página después del desplazamiento

        # Obtener nuevamente todos los elementos que contienen los datos de los productos después del desplazamiento
        shoes = driver.find_elements(By.CLASS_NAME, 'grid-item')
        # Crear una lista para almacenar el HTML de cada elemento
        for shoe in shoes:
            shoes_html.append(shoe.get_attribute('outerHTML'))
        scrolls = 3
        variableHeight = 2500
        time.sleep(0.5)


        # Buscar el botón "Siguiente"
        next_button = driver.find_element(By.XPATH, '//a[@data-auto-id="plp-pagination-next"]')
        # Hacer clic en el botón "Siguiente"
        next_button.click()
        # Esperar un breve período de tiempo para que se cargue la siguiente página
        time.sleep(3)
except:
    print("No se encontró el botón 'Siguiente' o ya no está presente.")

finally:
    driver.quit()

No se encontró el botón 'Siguiente' o ya no está presente.


Una vez ejecutado las dos veces que mencionamos, una para la categoría de hombres y otra para la categoría de mujeres, podemos visualizar la cantidad de registros que tenemos.

In [10]:
len(shoes_html)

2826

Ahora, con el uso de BeautifulSoup, extraemos solo la información que necesitamos y la vamos ingresando en otros arreglos particulares que serán, al final, las columnas de nuestro archivo CSV final. Nótese que si no se encuentra un dato, se maneja la situación asignándole un valor por defecto. En este caso, los atributos encontrados y almacenados en el dataset fueron:

- Id del producto
- Nombre del modelo
- Precio
- Categoría
- Número de Colores Disponibles

In [11]:
for shoe_html in shoes_html:
    shoeSoup = BeautifulSoup(shoe_html, 'html.parser')

    grid_item_div = shoeSoup.find('div', class_='grid-item')
    shoe_name_tag = shoeSoup.find('p', class_="glass-product-card__title")
    shoe_price_tag = shoeSoup.find('div', class_="gl-price-item notranslate")
    shoe_priceDiscount_tag = shoeSoup.find('div', class_="gl-price-item gl-price-item--sale notranslate")
    shoe_category_tag = shoeSoup.find('p', class_="glass-product-card__category")
    shoe_nColors_tag = shoeSoup.find('span', class_="dark-grey___6ysQv")

    if shoe_name_tag:
        shoe_id = grid_item_div.get('data-grid-id')
        shoe_name = shoe_name_tag.text.strip()
        shoe_price = shoe_price_tag.text.strip() if shoe_price_tag else shoe_priceDiscount_tag.text.strip() if shoe_priceDiscount_tag  else "Unknown"
        shoe_category = shoe_category_tag.text.strip() if shoe_category_tag else "Unknown"

        try:
            shoe_nColors = int(shoe_nColors_tag.text.split()[0])
        except Exception as e:
            shoe_nColors = "1"

        shoes_ids.append(shoe_id)
        shoes_names.append(shoe_name)
        shoes_prices.append(shoe_price)
        shoes_category.append(shoe_category)
        shoes_nColors.append(shoe_nColors)
        shoes_gender.append(gender)
    else:
        continue

En este punto, regresamos al paso donde definimos la fuente y cambiamos a la otra opción, en este caso, la sección de calzado para 'Mujer'. Luego, ejecutamos nuevamente hasta este punto.

---

Finalmente creamos el Dataset con todos los datos que tenemos.

In [12]:
# Crear un DataFrame con los datos de los tenis
data = {'id': shoes_ids,'shoe_name': shoes_names, 'shoe_price': shoes_prices, 'shoe_category': shoes_category, 'shoe_nColors': shoes_nColors, 'shoe_genre': shoes_gender}
df = pd.DataFrame(data)

# Guardar el DataFrame en un archivo CSV
df.to_csv('adidas_RAW.csv', index=False, encoding='utf-8-sig')
print("Los datos se han guardado exitosamente en 'adidas_RAW.csv'")

Los datos se han guardado exitosamente en 'adidas_RAW.csv'


---
---


### Codigo extra que se uso para pruebas

In [ ]:
driver.close()

In [ ]:
# for shoe_html in shoes_html:
#     shoeSoup = BeautifulSoup(shoe_html, 'html.parser')

#     shoe_name_tag = shoeSoup.find('p', class_="glass-product-card__title")
#     shoe_price_tag = shoeSoup.find('div', class_="gl-price-item notranslate")
#     shoe_priceDiscount_tag = shoeSoup.find('div', class_="gl-price-item gl-price-item--sale notranslate")
#     shoe_category_tag = shoeSoup.find('p', class_="glass-product-card__category")
#     shoe_nColors_tag = shoeSoup.find('span', class_="dark-grey___6ysQv")

#     if shoe_name_tag:
#         grid_item_div = shoeSoup.find('div', class_='grid-item')
#         print(grid_item_div.get('data-grid-id'))
#     else:
#         continue

In [ ]:
# # try:    
#     # Definir la cantidad de desplazamientos y el tiempo de espera entre cada uno
# scrolls = 4
# variableHeight = 2000

# # Iniciar el bucle para hacer clic en el botón "Siguiente" hasta que no esté presente
# # while True:
# try:
#     # Buscar el botón "Siguiente"
#     adiClub_button = driver.find_element(By.ID, 'gl-modal__close-mf-account-portal')
#     # Hacer clic en el botón "Siguiente"
#     adiClub_button.click()
# except Exception as e:
#     pass

# driver.execute_script("return document.body.scrollHeight")
# for i in range(scrolls):
#     # Desplazarse hacia abajo
#     driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight + {i * variableHeight});")
#     # Esperar un breve período de tiempo
#     time.sleep(1)
#     # Obtener la nueva altura de la página después del desplazamiento
# # Obtener nuevamente todos los elementos que contienen los datos de los productos después del desplazamiento
# shoes = driver.find_elements(By.CLASS_NAME, 'grid-item')
# scrolls = 3
# variableHeight = 2500

#         # # Buscar el botón "Siguiente"
#         # next_button = driver.find_element(By.XPATH, '//a[@data-auto-id="plp-pagination-next"]')
#         # # Hacer clic en el botón "Siguiente"
#         # next_button.click()
#         # # Esperar un breve período de tiempo para que se cargue la siguiente página
#         # time.sleep(2)
# # except:
# #     print("No se encontró el botón 'Siguiente' o ya no está presente.")

# # finally:
# #     # driver.quit()

In [ ]:
# # try:    
#     # Definir la cantidad de desplazamientos y el tiempo de espera entre cada uno
# scrolls = 4
# variableHeight = 2000

#     # Iniciar el bucle para hacer clic en el botón "Siguiente" hasta que no esté presente
#     # while True:
# try:
#     # Buscar el botón "Siguiente"
#     adiClub_button = driver.find_element(By.ID, 'gl-modal__close-mf-account-portal')
#     # Hacer clic en el botón "Siguiente"
#     adiClub_button.click()
# except Exception as e:
#     pass


# driver.execute_script("return document.body.scrollHeight")
# for i in range(scrolls):
#     # Desplazarse hacia abajo
#     driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight + {i * variableHeight});")
#     # Esperar un breve período de tiempo
#     time.sleep(1)
#     # Obtener la nueva altura de la página después del desplazamiento

# # Obtener nuevamente todos los elementos que contienen los datos de los productos después del desplazamiento
# shoes = driver.find_elements(By.CLASS_NAME, 'grid-item')
# # Crear una lista para almacenar el HTML de cada elemento
# for shoe in shoes:
#     shoes_html.append(shoe.get_attribute('outerHTML'))
# scrolls = 3
# variableHeight = 2500
# time.sleep(0.5)


# # # Buscar el botón "Siguiente"
# # next_button = driver.find_element(By.XPATH, '//a[@data-auto-id="plp-pagination-next"]')
# # # Hacer clic en el botón "Siguiente"
# # next_button.click()
# # # Esperar un breve período de tiempo para que se cargue la siguiente página
# # time.sleep(3)
# # except:
# #     print("No se encontró el botón 'Siguiente' o ya no está presente.")

# # finally:
# #     driver.quit()